Importando Bibliotecas
Impotando o arquivo diamonds.csv

In [ ]:
# processamento de dados, algebra linear
import numpy as np 
import pandas as pd

# visualização de dados
import seaborn as sns
import matplotlib.pyplot as plt

# impime os arquivos
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
# carrega o arquivo e exibe 3 amostras
df = pd.read_csv('../input/diamonds/diamonds.csv')
df.head(20)



In [ ]:
# exibe as 5 primeiras linhas
df.head(5)

In [ ]:
# Trocando as variáveis de texto por numéricas
df['color'] = df['color'].replace(['J','I','H','G','F','E','D'],[1,2,3,4,5,6,7])
df['cut'] = df['cut'].replace(['Fair','Good','Very Good','Premium','Ideal'],[1,2,3,4,5])
df['clarity'] = df['clarity'].replace(['I3','I2','I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF'],[1,2,3,4,5,6,7,8,9,10])

In [ ]:
# exibe as 5 primeiras linhas dados alfabeticos trocados por números
df.head(5)

Procurando por valores nulos e verificando os tipos de dados da base de dados.

In [ ]:
df.info()

In [ ]:
# Removendo a coluna Unnamed:0 - Pois aparentemente é um contador
dfn = df.drop(['Unnamed: 0'], axis = 1)
dfn.head(5)


In [ ]:
# Exibindo a descrição da base de dados
dfn.describe()

Verificando a correlação entre os dados

In [ ]:
# Verificando a correlação entre os dados
correlacoes = dfn.corr()
plt.figure(figsize=(11,8))
sns.heatmap(correlacoes, annot=True, fmt='.2f', cmap='coolwarm')
plt.show()

In [ ]:
dfn.hist(figsize=(10,8))
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
sns.scatterplot(data=dfn, x="carat", y="price")
plt.show()

Nota-se uma alta correlação entre o carat e as variáveis x,y e z, mas faz sentido pois o carat é o "peso" do diamante então justifica.

Nota-se uma correlação forte entre o carat e o preço, que também faz sentido.

Investigando melhor a relação entre carat, price e cut,	color, clarity que não aparecem na correlação	

In [ ]:
plt.figure(figsize=(16,8))
sns.scatterplot(data=dfn, x="carat", y="price", hue="color",size="cut", style="clarity", sizes=(10,250))
plt.show()

In [ ]:
##sns.set_theme(style="ticks")
sns.pairplot(data=dfn)
##plot.show()

In [ ]:
#Verificando cut vs price
sns.catplot(x="cut", y="price", kind="box", data=df);


In [ ]:
#Verificando color vs price
sns.catplot(x="color", y="price", kind="box", data=dfn);

In [ ]:
#Verificando clarity vs price
sns.catplot(x="clarity", y="price", kind="box", data=dfn);

In [ ]:
#Verificando carat vs price e clarity
plt.figure(figsize=(15,6))
sns.lineplot(x="carat", y="price", hue="clarity", palette='coolwarm', data=dfn);


In [ ]:
#Verificando carat vs price / clarity  e color 
plt.figure(figsize=(15,6))
sns.lineplot(x="carat", y="price", hue="clarity", style="color", palette='coolwarm', data=dfn);


## Carregando as Metricas

In [ ]:
# métricas
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
# variável de resultado final
# será armazenado o resultado de todos experimentos
experiment = {}
# treinamento, test split
from sklearn.model_selection import train_test_split


## Separando a coluna "price" que será inferida pelo sistema

In [ ]:
# recupera os valores (X), e as classes (Y)
X = dfn.drop('price', axis=1)
Y = dfn['price']


## Separando os conjuntos X e Y de treino e teste

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=6)

In [ ]:
print('treinamento:', len(y_train))
print('teste      :', len(y_test))

## Regressão KNN

In [ ]:
# regressor
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
# model1 é o modelo criado pelo regressor KNN
model1 = KNeighborsRegressor(n_neighbors=3,metric='euclidean')
model1.fit(X_train,y_train)


In [ ]:
# ypred é o modelo predito
y_pred = model1.predict(X_test)


In [ ]:
# Calculando as metricas para o KNN
# R Square Error
r2 = r2_score(y_test,y_pred)

# Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test,y_pred)

# Mean Square Error (MSE)
mse = mean_squared_error(y_test,y_pred, squared=True)

# Root Mean Square Error (RMSE)
rmse = mean_squared_error(y_test,y_pred, squared=False)

In [ ]:
# Imprimindo as metricas e armazenando no vetor experimet
experiment['KNN'] = {'R2':r2, 'MAE':mae, 'MSE':mse, 'RMSE':rmse}

print('R2  :',r2)
print('MAE :',mae)
print('MSE :',mse)
print('RMSE:',rmse)

## Regressão Random Forest

In [ ]:
# regressor
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# model5 é o modelo criado pela Random Forrest
model5 = RandomForestRegressor(n_estimators=10, random_state=26)
model5.fit(X_train, y_train)

In [ ]:
# Y_pred é o modelo predito pelo model5 random forrest
y_pred = model5.predict(X_test)

In [ ]:
# calculo das metricas para o Random Forrest
r2 = r2_score(y_test,y_pred)
mae = mean_absolute_error(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred, squared=True)
rmse = mean_squared_error(y_test,y_pred, squared=False)

In [ ]:
# imprimindo as metricas para o Random Forrest e atualizando o vetor experiment
experiment['Random Forest'] = {'R2':r2, 'MAE':mae, 'MSE':mse, 'RMSE':rmse}

print('R2  :',r2)
print('MAE :',mae)
print('MSE :',mse)
print('RMSE:',rmse)

In [ ]:
# imprimindo o resumo dos experimentos
pd.DataFrame(experiment).T

## Simulando a entrada de dados novos nos modelos de regressão

In [ ]:
# escolhendo o primeiro diamante do conjunto de teste
X_test.head(1)

In [ ]:
# a resposta do conjunto de teste do primeiro diamante
y_test.head(1)

In [ ]:
## inserindo estes dados no modelo do KNN como um novo diamante
data2 = [{"carat": 0.53, "cut": 3, "color": 7, "clarity": 5, "depth": 61.2, 
          "table": 54.0, "x": 5.23, "y": 5.26, "z": 3.21}]
dfu = pd.DataFrame(data2)
y_predu = model1.predict(dfu)
print(y_predu)



In [ ]:
## inserindo estes dados no modelo do Random Forrest como um novo diamante
data2 = [{"carat": 0.53, "cut": 3, "color": 7, "clarity": 5, "depth": 61.2, 
          "table": 54.0, "x": 5.23, "y": 5.26, "z": 3.21}]
dfu = pd.DataFrame(data2)
y_predu = model5.predict(dfu)
print(y_predu)

De fato o modelo do Random Forrest foi o que mais se aproximou da resposta do conjunto de teste.

## Outras simulações que não foram para o relatório.

# Regrassão Linear

In [ ]:
# regressor
from sklearn.linear_model import LinearRegression

In [ ]:
model2 = LinearRegression()
model2.fit(X_train, y_train)

In [ ]:
y_pred = model2.predict(X_test)

In [ ]:
r2 = r2_score(y_test,y_pred)
mae = mean_absolute_error(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred, squared=True)
rmse = mean_squared_error(y_test,y_pred, squared=False)

In [ ]:
experiment['Linear Regression'] = {'R2':r2, 'MAE':mae, 'MSE':mse, 'RMSE':rmse}

print('R2  :',r2)
print('MAE :',mae)
print('MSE :',mse)
print('RMSE:',rmse)

# Support Vector Machines (SVM)

In [ ]:
# regressor
from sklearn.svm import SVR


In [ ]:
model3 = SVR()
model3.fit(X_train, y_train)

In [ ]:
y_pred = model3.predict(X_test)

In [ ]:
r2 = r2_score(y_test,y_pred)
mae = mean_absolute_error(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred, squared=True)
rmse = mean_squared_error(y_test,y_pred, squared=False)

In [ ]:
experiment['SVM'] = {'R2':r2, 'MAE':mae, 'MSE':mse, 'RMSE':rmse}

print('R2  :',r2)
print('MAE :',mae)
print('MSE :',mse)
print('RMSE:',rmse)

# Árvore de Decisão

In [ ]:
# regressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree

In [ ]:
model4 = DecisionTreeRegressor(random_state=26)
model4.fit(X_train, y_train)

In [ ]:
y_pred = model4.predict(X_test)

In [ ]:
r2 = r2_score(y_test,y_pred)
mae = mean_absolute_error(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred, squared=True)
rmse = mean_squared_error(y_test,y_pred, squared=False)

In [ ]:
experiment['Decision Tree'] = {'R2':r2, 'MAE':mae, 'MSE':mse, 'RMSE':rmse}

print('R2  :',r2)
print('MAE :',mae)
print('MSE :',mse)
print('RMSE:',rmse)

In [ ]:
text_representation = tree.export_text(model4)
print(text_representation)

In [ ]:
##fig = plt.figure(figsize=(25,20))
##_ = tree.plot_tree(model4,filled=True)

In [ ]:
# imprimindo o resumo dos experimentos
pd.DataFrame(experiment).T